In [1]:
import datetime
import pickle
import os
import random
import sympy as sp

In [2]:
class Pet:
    """A class representing a virtual pet.

    Attributes:
        ALLOWED_TYPES (list of str): Class-level list of allowed pet types.
        name (str): The pet's name.
        pet_type (str): The type of the pet; must be in ALLOWED_TYPES.
        hunger (int): Hunger level of the pet (0-100).
        happiness (int): Happiness level of the pet (0-100).
        strength (int): Strength level of the pet (0-100).
        created_at (datetime): Timestamp when the pet was created.
        last_saved (datetime): Timestamp of last save/pickle operation.
    """
    ALLOWED_TYPES = [
        "Dog",
        "Cat",
        "Dragon",
        "Rock",
        "Star",
        "Philosopher"
        ]  # Class variable for pet tyoe

    def __init__(self, name, pet_type):
        """Initializes a Pet instance with the given name and type.

        Args:
            name (str): Name of the pet.
            pet_type (str): Type of the pet. Must be in ALLOWED_TYPES.

        Raises:
            ValueError: If pet_type is not in ALLOWED_TYPES.
        """
        if pet_type not in Pet.ALLOWED_TYPES:
            raise ValueError(f"Invalid Pet Type! Choose from: {Pet.ALLOWED_TYPES}")
        self.name = name
        self.pet_type = pet_type
        self.hunger = 70
        self.happiness = 70
        self.strength = 70
        self.created_at = datetime.datetime.now()  # when pet is created
        self.last_saved = self.created_at  # updates everytime the pet is pickled

    def get_age(self):
        """Calculates the pet's age in days and hours.

        Returns:
            tuple: (days, hours) representing the pet's age.
        """
        now = datetime.datetime.now()
        age = now - self.created_at
        days = age.days
        hours = age.seconds // 3600
        return days, hours

    def feed(self, amount):
        """Increases the pet's hunger level.

        Args:
            amount (int): Amount to increase hunger by.
        """
        self.hunger += amount
        if self.hunger > 100:
            self.hunger = 100

    def play(self, amount):
        """Increases the pet's happiness level.

        Args:
            amount (int): Amount to increase happiness by.
        """
        self.happiness += amount
        if self.happiness > 100:
            self.happiness = 100

    def train(self, amount):
        """Increases the pet's strength level.

        Args:
            amount (int): Amount to increase strength by.
        """
        self.strength += amount
        if self.strength > 100:
            self.strength = 100

    def decay(self):
        """Applies decay to hunger, happiness, and strength over time.

        Decay is calculated based on the time elapsed since the last save.
        """
        now = datetime.datetime.now()
        elapsed_hours = (now - self.last_saved).total_seconds() / 3600

        elapsed_hours = (
            elapsed_hours * 12
        )  # 12 intervals per hour -> 1 interval = 5 minutes

        # Decay rates per hour
        HUNGER_DECAY = 2
        HAPPINESS_DECAY = 1.5
        STRENGTH_DECAY = 1

        # Apply decay
        self.hunger = max(round(self.hunger - elapsed_hours * HUNGER_DECAY), 0)
        self.happiness = max(round(self.happiness - elapsed_hours * HAPPINESS_DECAY), 0)
        self.strength = max(round(self.strength - elapsed_hours * STRENGTH_DECAY), 0)

    def status(self):
        """Prints the current status and age of the pet."""
        days, hours = self.get_age()
        print(f"\nName: {self.name}")
        print(f"Type: {self.pet_type}")
        print(f"Hunger: {self.hunger}")
        print(f"Happiness: {self.happiness}")
        print(f"Strength: {self.strength}")
        print(f"Birthday: {self.created_at.strftime('%B %d, %Y')}")
        print(f"Age: {days} days, {hours} hours")

In [3]:
def save_pet(pet, player_name):
    """Saves a Pet object to a file using pickle.

    The filename is based on the player name and pet name.

    Args:
        pet (Pet): The pet instance to save.
        player_name (str): The name of the player.

    Side Effects:
        Creates or overwrites a pickle file with the pet's data.
        Updates the pet's last_saved timestamp.
        Prints a confirmation message.
    """
    save_file = f"{player_name}_{pet.name}_pet.pkl"
    pet.last_saved = datetime.datetime.now()

    with open(save_file, "wb") as file:
        pickle.dump(pet, file)
    print(f"💾 Game saved as {save_file}!")


def load_pet(player_name, pet_name):
    """Loads a Pet object from a pickle file and applies decay.

    Args:
        player_name (str): Name of the player who owns the pet.
        pet_name (str): Name of the pet to load.

    Returns:
        Pet | None: Returns the loaded Pet object if found; otherwise, None.

    Side Effects:
        Applies decay to the pet based on elapsed time.
        Prints the pet's status if successfully loaded.
    """
    save_file = f"{player_name}_{pet_name}_pet.pkl"
    if os.path.exists(save_file):
        with open(save_file, "rb") as file:
            pet = pickle.load(file)
        pet.decay()
        print(f"{pet_name} the {pet.pet_type} is now active!")
        pet.status()
        return pet
    else:
        print("❌ Sorry, Pet not found!")
        return None

In [4]:
x = sp.symbols("x")


def pet_challenge(level):
    """Generates a math challenge for a virtual pet game.

    Challenges vary by difficulty level:
        1 - Easy: simple addition
        2 - Average: quadratic equation
        3 - Difficult: polynomial differentiation

    Args:
        level (int): Difficulty level (1, 2, or 3).

    Returns:
        tuple: (question (str), correct_answer, points (int))
            - question: The text of the challenge.
            - correct_answer: The correct solution (int, list, or sympy expression).
            - points: Points awarded for correct answer.
            Returns (None, None, 0) if level is invalid.
    """
    if level not in [1, 2, 3]:
        print("⚠️ Invalid Level! Choose 1, 2, or 3")
        return None, None, 0

    if level == 1:  # Easy
        num1 = random.randint(1, 100)
        num2 = random.randint(1, 100)
        question = f"\nSolve: {num1} + {num2} = ?"
        correct_answer = num1 + num2
        pts = 2

    elif level == 2:  # Average
        r1 = random.randint(1, 10)
        r2 = random.randint(1, 10)
        coef_a = random.randint(1, 5)
        coef_b = -coef_a * (r1 + r2)
        coef_c = coef_a * r1 * r2

        quadratic = coef_a * x**2 + coef_b * x + coef_c
        roots = sp.solve(quadratic, x)
        question = f"\nSolve the quadratic: {quadratic} = 0"
        correct_answer = roots
        pts = 5

    elif level == 3:  # Difficult
        num_terms = random.randint(2, 4)
        func = 0
        for _ in range(num_terms):
            coef = random.randint(1, 10)
            exp = random.randint(2, 10)
            func += coef * x**exp

        derivative = sp.diff(func, x)
        question = f"\nDifferentiate: f(x) = {func}"
        correct_answer = derivative
        pts = 10

    return question, correct_answer, pts

In [5]:
def action_challenge(pet, action, attribute):
    """Conducts a challenge to improve a pet's attribute.

    Presents a math challenge to the player. Success grants points to
    the specified pet attribute.

    Args:
        pet (Pet): The pet instance to interact with.
        action (str): The action being performed (e.g., "feed", "play", "train").
        attribute (str): The attribute that will be affected by points
            (e.g., "hunger", "happiness", "strength").

    Returns:
        int: Points earned from the challenge (0 if player gives up or inputs invalid level).
    """
    x = sp.symbols("x")
    print(f"\nTime to {action} {pet.name}!")
    print(f"Solve the challenge to get {action} points.")
    print(f"\nChoose level: ")
    print("1. Easy - +2")
    print("2. Average - +5")
    print("3. Difficult - +10")

    try:
        level = int(input("\nEnter level number: ").strip())
        if level not in [1, 2, 3]:
            raise ValueError("Invalid level! Choose 1, 2, or 3")

        question, correct_answer, pts = pet_challenge(level)
        print(question)

        correct = False
        while not correct:
            player_input = input("Your answer (or type 'q' to give up): ").strip()

            if player_input.lower() == "q":
                print("You gave up! No points earned.")
                return 0

            # Easy level
            if level == 1:
                try:
                    player_input_int = int(player_input)
                    if player_input_int == correct_answer:
                        correct = True
                        print(f"🎉 Correct! {pet.name} now has {pts} {attribute} points.")
                    else:
                        print("❌ Wrong answer! Try again or type 'q' to give up.")
                except:
                    print("❌ Please enter a valid number or 'q' to give up.")

            # Average level
            elif level == 2:
                try:
                    player_roots = [int(x.strip()) for x in player_input.split(",")]
                    if set(player_roots) == set(correct_answer):
                        correct = True
                        print(f"🎉 Correct! {pet.name} now has {pts} {attribute} points.")
                    else:
                        print("❌ Wrong answer! Try again or type 'q' to give up.")
                except:
                    print(
                        "❌ Please enter roots as numbers separated by commas, or 'q' to give up."
                    )

            # Difficult level
            elif level == 3:
                try:
                    player_deriv = sp.sympify(player_input)
                    if sp.simplify(player_deriv - correct_answer) == 0:
                        correct = True
                        print(f"🎉 Correct! {pet.name} now has {pts} {attribute} points.")
                    else:
                        print("❌ Wrong answer! Try again or type 'q' to give up.")
                except:
                    print(
                        "❌ Invalid derivative format! Try again or type 'q' to give up."
                    )

        return pts

    except ValueError as error_message:
        print(error_message)
        return 0

In [ ]:
print(
    r"""
========================================
         🐾 Welcome to NicoPets! 🐾
   Feed, Play, Train, and Grow Your Pet!
========================================
"""
)


# Ask for player name
player_name = input("Enter your player name: ").strip()

# Ask for name
pet_name = input("Enter your pet's name: ").strip()

# Check if pet exists
my_pet = load_pet(player_name, pet_name)

if my_pet is None:
    print(f"\n creating a new pet for {player_name}...")

    # Ask for type
    print(f"Choose your pet type: {Pet.ALLOWED_TYPES}")
    pet_type = input("Enter your pet's type: ").strip()

    # Validate
    while pet_type not in Pet.ALLOWED_TYPES:
        print("❌ Invalid Pet Type! Please choose again")
        print(f"Options: {Pet.ALLOWED_TYPES}")
        pet_type = input("Enter your pet's type: ").strip()

    # Create_pet
    my_pet = Pet(pet_name, pet_type)
    print(f"\n🎉 You now have {my_pet.name} the {my_pet.pet_type}!")
    my_pet.status()

while True:
    print("\nWhat do you want to do?")
    print("1. Feed")
    print("2. Play")
    print("3. Train")
    print("4. Check Status")
    print("5. Save & Exit")

    choice = int(input("\nEnter your choice (1-5): ").strip())

    if choice == 1:
        amount = action_challenge(my_pet, "feed", "hunger")
        my_pet.feed(amount)
        #print(f"🎉Correct! {my_pet.name} now has {my_pet.hunger} hunger points.")
    elif choice == 2:
        amount = action_challenge(my_pet, "play with", "happiness")
        my_pet.play(amount)
        #print(f"🎉Correct! {my_pet.name} now has {my_pet.happiness} happiness points.")
    elif choice == 3:
        amount = action_challenge(my_pet, "train", "strength")
        my_pet.train(amount)
        #print(f"🎉Correct! {my_pet.name} now has {my_pet.strength} strength points.")
    elif choice == 4:
        my_pet.status()
    elif choice == 5:
        save_pet(my_pet, player_name)  # Save on exit
        print("Goodbye!")
        break
    else:
        print("❌ Invalid choice. Try again.")


         🐾 Welcome to NicoPets! 🐾
   Feed, Play, Train, and Grow Your Pet!



Enter your player name:  Nico
Enter your pet's name:  Nico


Nico the Rock is now active!

Name: Nico
Type: Rock
Hunger: 0
Happiness: 0
Strength: 0
Birthday: August 24, 2025
Age: 11 days, 1 hours

What do you want to do?
1. Feed
2. Play
3. Train
4. Check Status
5. Save & Exit



Enter your choice (1-5):  2



Time to play with Nico!
Solve the challenge to get play with points.

Choose level: 
1. Easy - +2
2. Average - +5
3. Difficult - +10


...


Differentiate: f(x) = 6*x**8 + 11*x**7 + 2*x**2


Your answer (or type 'q' to give up):  48*x**7 + 77*x**6 + 4*x


🎉 Correct! Nico now has 10 happiness points.

What do you want to do?
1. Feed
2. Play
3. Train
4. Check Status
5. Save & Exit



Enter your choice (1-5):  4



Name: Nico
Type: Rock
Hunger: 0
Happiness: 10
Strength: 0
Birthday: August 24, 2025
Age: 11 days, 1 hours

What do you want to do?
1. Feed
2. Play
3. Train
4. Check Status
5. Save & Exit
